In [14]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

### Load Cleaned Data from load_and_clean

In [15]:
data = pd.read_pickle("data/cleaned_data.pkl")


In [16]:
data.head(1)

,Unnamed: 0.1,Unnamed: 0,testID,rep,profileID,recordedUTC,Team,Category,BODY_WEIGHT_N,BODY_WEIGHT,...,TOTAL_WORK,VELOCITY_AT_PEAK_POWER,CMJ_STIFFNESS,ECCENTRIC_UNLOADING_IMPULSE,id_rep,time_series,total_force,initial_value,normalized_time_series,downsampled_time_series
0,0,0,f6b0b4f3-ccaa-488c-b71b-2a9d6fd8c359,1,eb037fd9-1b41-4bee-ade5-079f3d3012f8,2023-01-06 23:31:43.937000+00:00,Acro and Tumbling,Sport,NaN,NaN,...,352.793276,1.889667,result\nCMJ_STIFFNESS 8803.572568\nCMJ_STIF...,result\nECCENTRIC_UNLOADING_IMPULSE -42.7836...,f6b0b4f3-ccaa-488c-b71b-2a9d6fd8c359_1,"[[259.619805, 259.820513], [257.619805, 261.82...","[519.4403179999999, 519.4403179999999, 516.440...",519.440318,"[1.0, 1.0, 0.994224553050578, 0.99422455305057...","[0.9997689723968506, 0.9933004975318909, 0.982..."


In [17]:
processed_data = np.array(data["downsampled_time_series"].tolist())
processed_data = torch.tensor(processed_data, dtype=torch.float32)  

In [19]:
len(processed_data[0])

100

### Create Autoencoder

In [20]:
# Create Autoencoder 
class Autoencoder(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        self.latent_dim = latent_dim
        
        self.encoder = nn.Sequential(
            nn.Linear(100, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, self.latent_dim)  
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(self.latent_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 100),
            # nn.Sigmoid()
        )
    
    def encode(self, x):
        return self.encoder(x)  
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    

### Use Processed/Downsampled Code to Train Model

In [21]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(processed_data, test_size=0.2, random_state=42)

In [22]:
latent_dim = 16
autoencoder = Autoencoder(latent_dim).to(device)
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.001)
loss_function = nn.MSELoss()  # Mean Squared Error for reconstruction loss

train_data = train_data.to(device)
test_data = test_data.to(device)

In [23]:
from torch.utils.data import TensorDataset, DataLoader
dataset = TensorDataset(train_data, train_data)


dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

### Train Model

In [25]:
num_epochs = 50
for epoch in range(num_epochs):
    autoencoder.train()  # Set model to training mode
    
    for x, y in dataloader:
        
        

        optimizer.zero_grad()  # Reset gradients
    

        outputs = autoencoder(x)  
        loss = loss_function(outputs, x)  
    
        loss.backward() 
        optimizer.step()  


    autoencoder.eval()
    with torch.no_grad():
        test_outputs = autoencoder(test_data)
        test_loss = loss_function(test_outputs, test_data)

    if epoch % 1 == 0:
        print(f"Epoch [{epoch}/{num_epochs}], Train Loss: {loss.item():.4f}, Test Loss: {test_loss.item():.4f}")

Epoch [0/50], Train Loss: 0.0119, Test Loss: 0.0054
Epoch [1/50], Train Loss: 0.0032, Test Loss: 0.0051
Epoch [2/50], Train Loss: 0.0056, Test Loss: 0.0052
Epoch [3/50], Train Loss: 0.0042, Test Loss: 0.0048
Epoch [4/50], Train Loss: 0.0041, Test Loss: 0.0042
Epoch [5/50], Train Loss: 0.0020, Test Loss: 0.0036
Epoch [6/50], Train Loss: 0.0019, Test Loss: 0.0035
Epoch [7/50], Train Loss: 0.0019, Test Loss: 0.0036
Epoch [8/50], Train Loss: 0.0028, Test Loss: 0.0034
Epoch [9/50], Train Loss: 0.0033, Test Loss: 0.0034
Epoch [10/50], Train Loss: 0.0092, Test Loss: 0.0037
Epoch [11/50], Train Loss: 0.0035, Test Loss: 0.0033
Epoch [12/50], Train Loss: 0.0029, Test Loss: 0.0033
Epoch [13/50], Train Loss: 0.0037, Test Loss: 0.0034
Epoch [14/50], Train Loss: 0.0029, Test Loss: 0.0034
Epoch [15/50], Train Loss: 0.0053, Test Loss: 0.0034
Epoch [16/50], Train Loss: 0.0055, Test Loss: 0.0034
Epoch [17/50], Train Loss: 0.0054, Test Loss: 0.0035
Epoch [18/50], Train Loss: 0.0028, Test Loss: 0.0032
Epo

### Save Model to 'autoencoder_normalized.pth'

In [26]:
model_path = "autoencoder_stochastic.pth"

torch.save({
    'model_state_dict': autoencoder.state_dict(),
    'optimizer_state_dict': optimizer.state_dict()
}, model_path)

print(f"Model saved to {model_path}")

Model saved to autoencoder_stochastic.pth
